均值平滑计算点领域的平均值，利用图像积分实现快速均值平滑，比高斯计算快一些？高斯平滑利用高斯滤波器进行滤波平滑。

In [19]:
import numpy as np
import cv2

image = cv2.imread("img3.jpg",0)
#均值平滑
dst = cv2.blur(image,(10,10))
cv2.imshow("src",image)
cv2.imshow("dst",dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

原理部分

In [5]:

def integral(image):
    rows,cols = image.shape
    #行积分计算
    inteImageC = np.zeros((rows,clos),np.float32)
    for r in xrange(rows):
        for c in xrange(cols):
            if c == 0:
                inteImageC[r][c] = image[r][c]
            else:
                inteImageC[r][c] = inteImageC[r][c-1] + image[r][c]
    #列积分计算
    inteImage = np.zeros(image.shape,np.float32)
    for c in xrange(cols):
        for r in xrange(rows):
            if r == 0:
                inteImage[r][c] = inteImageC[r][c]
            else:
                inteImage[r][c] = inteImage[r-1][c] + inteImageC[r][c]
    #对上边，左边进行补零操作，省去平滑中判断边界问题
    inteImage_0 = np.zeros((rows+1,cols+1),np.float32)
    inteImage_0[1:rows+1,1:cols+1] = inteImage
    return inteImage_0

#构建均值平滑函数fastMeanBlur,滑动窗口winSize均为奇数
#borderType边界扩充类型补零操作会造成黑色边界。理想扩充类型为镜像扩充
def fastMeanBlur(image,winSize,borderType = cv2.BORDER_DEFAULT):
    halfH = (winSize[0]-1)/2
    halfW = (winSize[1]-1)/2
    ratio = 1.0/(winSize[0]*winSize[1])
    #边界扩充
    paddImage = cv2.copyMakeBorder(image,halfH,halfH,halfW,halfW,borderType)
    #图像积分
    paddIntegral = integral(paddImage)
    #图像的高、宽
    rows,cols = image.shape
    #均值滤波后的结果
    meanBlurImage = np.zeros(image.shape,np.float32)
    r,c = 0,0
    for h in range(halfH,halfH+rows,1):
        for w in range(halfW,halfW+cols,1):
            meanBlurImage[r][c]=(paddIntegral[h+halfH+1][w+halfW+1]+paddIntegral[h-halfH][w-halfW]-paddIntegral[h+halfH+1][w-halfW]-paddIntegral[h-halfH][w+halfW+1])*ratio
            c+=1
        r+=1
        c=0
    return meanBlurImage

NameError: name 'cv2' is not defined